In [1]:
print("ok")

ok


In [2]:
%pwd

'c:\\Users\\Manas\\Desktop\\bot\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Manas\\Desktop\\bot'

In [5]:
import os
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
from groq import Groq

c:\Users\Manas\Anaconda3\envs\app\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [6]:
load_dotenv()
PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY = os.environ.get('GROQ_API_KEY')

In [7]:
assert PINECONE_API_KEY, "Pinecone API key not found!"
assert GROQ_API_KEY, "Groq API key not found!"

In [8]:
#Extract Data  
def load_pdf_file(data_path):
    loader = DirectoryLoader(data_path, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [9]:
extracted_data = load_pdf_file('Data/')
print("Documents loaded:", len(extracted_data))

Documents loaded: 28


In [49]:
#Splits data into text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks


In [11]:
text_chunks = text_split(extracted_data)
print("Length of Text Chunks:", len(text_chunks))

Length of Text Chunks: 185


In [12]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\Manas\AppData\Local\Temp\ipykernel_24452\3409896792.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [13]:
query_result = embeddings.embed_query("Hello world")
print("Embedding length:", len(query_result))

Embedding length: 384


In [14]:
pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "bot"

In [15]:
pc.create_index(
    name=index_name,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


PineconeApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'x-pinecone-api-version': '2024-07', 'X-Cloud-Trace-Context': '6d66c5995a68d57c132d78d79b094cd5', 'Date': 'Thu, 30 Jan 2025 18:52:36 GMT', 'Server': 'Google Frontend', 'Content-Length': '85', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"ALREADY_EXISTS","message":"Resource  already exists"},"status":409}


In [16]:
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings,
)


In [17]:
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)


In [18]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.invoke("Tell me about Gautam Buddha University?")
print("Retrieved Documents:", retrieved_docs)

Retrieved Documents: [Document(id='b47cd223-2332-4ffc-a434-6b9c436f20c5', metadata={'page': 0.0, 'page_label': '1', 'source': 'Data\\gbu_data.pdf'}, page_content='envisions to integrate our National Ethos, Rich Heritage and the Ancient Wisdom with the education and \nresearch to invoke high \nstandards of quality and innovativeness to be at par with international benchmarks. The \nUniversity is a world class center for excellence in the field to Research and Academics. \n \nGautam Buddha University has 455 acres of lush green campus with the built up area of'), Document(id='5458db82-3997-49fb-a9c2-dd8a5673f1e1', metadata={'page': 0.0, 'page_label': '1', 'source': 'Data\\gbu_data.pdf'}, page_content='envisions to integrate our National Ethos, Rich Heritage and the Ancient Wisdom with the education and \nresearch to invoke high \nstandards of quality and innovativeness to be at par with international benchmarks. The \nUniversity is a world class center for excellence in the field to Rese

In [19]:
client = Groq(api_key=GROQ_API_KEY)

In [21]:
completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "user", "content": "Tell me about Gautam Buddha University."}
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=True,
    stop=None,
)

print("\nGroq LLM Response:")
for chunk in completion:
    print(chunk.choices[0].delta.content or "", end="")



Groq LLM Response:
Gautam Buddha University (GBU) is a state university located in Greater Noida, Uttar Pradesh, India. It was established in 2008 by the Government of Uttar Pradesh through an Act of the State Legislature. The university is named after Gautam Buddha, a spiritual teacher from ancient India who founded Buddhism.

GBU is a research-oriented university that offers a wide range of undergraduate, postgraduate, and doctoral programs in various fields, including:

1. Engineering: The university has a strong engineering program with departments such as Computer Science and Engineering, Electronics and Communication Engineering, Mechanical Engineering, and Civil Engineering.
2. Management: GBU offers MBA, BBA, and other management programs through its School of Management.
3. Law: The university has a School of Law that offers LL.B, LL.M, and other law programs.
4. Humanities and Social Sciences: GBU offers programs in English, Hindi, History, Sociology, Psychology, and other h

In [51]:
def handle_query(query):
    query_result = embeddings.embed_query(query)
    retrieved_docs = retriever.invoke(query)

    if not retrieved_docs:
        print("No relevant information is available.")
        return

    context = "\n".join(doc.page_content for doc in retrieved_docs)

    system_prompt = {
        "role": "system",
        "content": (
            "Answer the user's query *only* based on the given information. "
            "Do NOT add any extra details beyond what is relevant. "
            "If the answer is not found, simply reply 'No relevant information available.' "
            "Be direct and concise in your response."
        )
    }

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            system_prompt,  
            {"role": "user", "content": f"{context}\n\nUser question: {query}"}
        ],
        temperature=0,  
        max_completion_tokens=256,  
        top_p=1,
        stream=True,
        stop=None,
    )

    response_text = ""

    for chunk in completion:
        if chunk.choices[0].delta.content:
            response_text += chunk.choices[0].delta.content

    response_text = response_text.strip()
    print(response_text)


In [52]:
handle_query("Tell me about gautam buddha university")

Gautam Buddha University has a 455-acre lush green campus and aims to integrate National Ethos, Rich Heritage, and Ancient Wisdom with education and research to achieve international standards. It is a world-class center for excellence in research and academics.


In [53]:
handle_query("Did you know gautam buddha university contact detail")

No relevant information available.
